In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# A Simple Calculator

This file shows the implementation of a trivial calculator using `Ply`.  The grammar for the language implemented by this parser is as follows:
$$
\begin{array}{lcl}
  \texttt{stmnt}   & \rightarrow & \;\texttt{expr}\; \texttt{';'} \\[0.2cm]   
  \texttt{expr}    & \rightarrow & \;\texttt{expr}\; \texttt{'+'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{product}                                  \\[0.2cm]
  \texttt{product} & \rightarrow & \;\texttt{product}\; \texttt{'*'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{factor}                                   \\[0.2cm]
  \texttt{factor}  & \rightarrow &   \texttt{'('} \; \texttt{expr} \;\texttt{')'}        \\
                   & \mid        & \;\texttt{NUMBER}                               \end{array}
$$

## Specification of the Scanner

In [ ]:
import ply.lex as lex

There is only a single token that need to be defined via a regular expression.  
The other tokens, namely the operators `+` and `*` and the parenthesis consist 
only of a single character and can therefore be defined as literals.

In [ ]:
tokens = [ 'NUMBER' ]

The token `NUMBER` specifies a fully featured *floating point number*.

In [ ]:
def t_NUMBER(t):
    r'0|[1-9][0-9]*'
    t.value = int(t.value)
    return t

In [ ]:
literals = ['+', '*', '(', ')', ';']

Blanks and tabulators are ignored.

In [ ]:
t_ignore  = ' \t\n'

Unkown characters are reported as lexical errors.

In [ ]:
def t_error(t):
    print(f"Illegal character '{t.value[0]}' at character number {t.lexer.lexpos} in line {t.lexer.lineno}.")
    t.lexer.skip(1)

The next line is necessary because we use `Ply` from a notebook.

In [ ]:
__file__ = 'main'

We generate the lexer.

In [ ]:
lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The *start variable* of our grammar is `statement`.

In [ ]:
start = 'stmnt'

In [ ]:
def p_stmnt_expr(p):
    "stmnt : expr ';'"
    print(p[1])

An *expr* is a sequence of *products* that are combined with the operator `+`.
The corresponding grammar rules are:
```
    expr : expr '+' product
         | product
         ;
```

In [ ]:
def p_expr_plus(p):
    "expr : expr '+' product"
    p[0] = p[1] + p[3]
    
def p_expr_product(p):
    "expr : product"
    p[0] = p[1]

A *product* is a sequence of *factors* that are combined with the operator `*`.
The corresponding grammar rules are:
```
    product : product '*' factor
            | factor
            ;
```

In [ ]:
def p_product_multiply(p):
    "product : product '*' factor"
    p[0] = p[1] * p[3]
       
def p_product_factor(p):
    "product : factor"
    p[0] = p[1]

A factor is either an expression in parenthesis or a number.
```
   factor : '(' expr ')'
          | NUMBER
          | IDENTIFIER
          ;
```
If an identifier is undefined, its value is `NaN`, which is short for *not a number*.

In [ ]:
def p_factor_group(p):
    "factor : '(' expr ')'"
    p[0] = p[2]

def p_factor_number(p):
    "factor : NUMBER"
    p[0] = p[1]

The method `p_error` is called if a syntax error occurs.  The argument `p` is the token that could not be read.  If `p` is `None` then the syntax error occurs at the end of the input.

In [ ]:
def p_error(p):
    if p:
        print(f"Syntax error at character number {p.lexer.lexpos} at token '{p.value}' in line {p.lexer.lineno}.")
    else:
        raise(f"Incomplete input.")

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an *obscure bug* where the parser generator tries to read an empty parse table.
We set `debug` to `True` so that the parse tables are dumped into the file `parser.out`.

In [ ]:
parser = yacc.yacc(write_tables=False, debug=True)

Let's look at the action table that is generated.

The parser is invoked by calling the method `yacc.parse(s)` where `s` is a string that is to be parsed.

In [ ]:
def main():
    while True:
        s = input('calc> ')
        if s == '':
            break
        yacc.parse(s)

In [ ]:
main()